# Using a Convolution Neural Network for Species Distribution Modeling
## Sample Code

In [14]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
import geopandas as gpd
import cartopy
import xarray as xr
import cartopy.crs as ccrs
import cmocean

from time import time
from sklearn.utils import Bunch
from sklearn.datasets import fetch_species_distributions
from sklearn import svm, metrics
from scipy.interpolate import griddata
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

Idea: Make raster layers out of species composition data and environmental layers
X is temp, salinity, oxygen, nitrate, chlorophyll, and pH + phytobase
1. Find data sources for X variable climatologies (need 2D grids over the same area) 
2. *VISUALIZE LAYERS*
3. Modify exising CNN species distribution code with my X and Y
4. Train and predict


### Open and plot environmental layers
- Check dimensions of data array
- Current layer: B-SOSE Dec. 31 2018 depth -55m

In [21]:
def so_ax():
  map_proj = ccrs.PlateCarree()
  fig = plt.figure(figsize=[10, 10])  # inches
  ax = plt.subplot(projection=map_proj)
  ax.set_extent([-180, 180, -90, -40], ccrs.PlateCarree())
  fig.subplots_adjust(bottom=0.05, top=0.95, left=0.04, right=0.95, wspace=0.02)
  return ax 

In [ ]:
phytobase = pd.read_csv('../../data/out/cnn_input_layers/phytobase_occurrence_classes.csv')

In [7]:
# Do the top 5 species account for most of the data
# Check top value counts when averaging over austral seasons
phytobase['class'].value_counts()
# phytobase.columns

Bacillariophyceae            7087
Dinophyceae                  3293
Prymnesiophyceae             1712
Coccolithophyceae             910
Cyanophyceae                  727
Haptophyta incertae sedis      35
Prasinophyceae                 16
Pyramimonadophyceae            13
Dictyochophyceae               10
Cryptophyceae                   1
Telonemea                       1
Chrysophyceae                   1
Name: class, dtype: int64

In [6]:
phytobase

,scientificname,longitude,latitude,depth,taxonRank,occurrenceStatus,phylum,class,timestamp,AphiaID
0,Coccopterum labyrinthus,144.4130,-42.6097,126.0,SPECIES,PRESENT,Chlorophyta,Prasinophyceae,2000-02-29,620590.0
1,Coccopterum labyrinthus,144.4130,-42.6097,17.0,SPECIES,PRESENT,Chlorophyta,Prasinophyceae,2000-02-29,620590.0
2,Coccopterum labyrinthus,144.4130,-42.6097,319.0,SPECIES,PRESENT,Chlorophyta,Prasinophyceae,2000-02-29,620590.0
3,Coccopterum labyrinthus,144.4130,-42.6097,36.0,SPECIES,PRESENT,Chlorophyta,Prasinophyceae,2000-02-29,620590.0
4,Coccopterum labyrinthus,144.4130,-42.6097,473.0,SPECIES,PRESENT,Chlorophyta,Prasinophyceae,2000-02-29,620590.0
...,...,...,...,...,...,...,...,...,...,...
13801,Octactis speculum,0.0333,-31.9667,75.0,SPECIES,PRESENT,Ochrophyta,Dictyochophyceae,1979-01-09,1310442.0
13802,Octactis speculum,0.0500,-40.0500,100.0,SPECIES,PRESENT,Ochrophyta,Dictyochophyceae,1979-01-09,1310442.0
13803,Octactis speculum,0.0000,-35.0000,100.0,SPECIES,PRESENT,Ochrophyta,Dictyochophyceae,1979-01-09,1310442.0
13804,Octactis speculum,0.0000,-35.0000,75.0,SPECIES,PRESENT,Ochrophyta,Dictyochophyceae,1979-01-09,1310442.0


In [12]:
# Define the input shape of your data
height, width, num_env_layers = 588, 2160, 5
input_shape = (height, width, num_env_layers + 1) # add 1 to account for the presence/absence layer

# Generate some dummy data for demonstration purposes
num_samples = 2 # TODO: modify this to be unique species number; thats the number of input layers
x_train = np.random.rand(num_samples, height, width, num_env_layers + 1)
y_train = np.random.randint(2, size=(num_samples, 1))

array([[[[3.81692958e-04, 9.44451931e-01, 2.28918070e-01,
          5.76076687e-01, 9.87453033e-01, 3.68049614e-01],
         [9.59479435e-01, 4.32918577e-01, 8.32726643e-01,
          7.58770993e-01, 5.18326914e-01, 7.14751232e-01],
         [2.82840722e-01, 3.03346720e-01, 8.12012564e-01,
          7.46725440e-01, 8.34383412e-01, 2.95116421e-01],
         ...,
         [1.37054114e-01, 2.74043080e-01, 4.12695964e-01,
          9.56921894e-01, 3.64912141e-01, 4.25498398e-01],
         [2.09943467e-01, 6.06696340e-01, 1.95839317e-01,
          2.13191856e-01, 2.27582774e-01, 3.46866799e-01],
         [1.12757587e-01, 8.80901277e-03, 1.01052106e-01,
          3.27784847e-01, 5.28183952e-01, 8.98983938e-02]],

        [[7.37998342e-01, 3.02452575e-01, 4.05707058e-01,
          9.94583028e-01, 7.73463094e-01, 4.13345284e-02],
         [6.83345578e-01, 7.80971228e-01, 3.20855449e-01,
          5.62694258e-01, 3.71555439e-01, 1.28853378e-01],
         [5.22443243e-01, 5.66928570e-01, 1.2615

In [3]:

# Only use 5 most commonly occurring classes of phytoplankton
# Bacillariophyceae            7087
# Dinophyceae                  3293
# Prymnesiophyceae             1712
# Coccolithophyceae             910
# Cyanophyceae                  727

# TODO: use DENEU paper to figure out x_train and y_train shapes
# load data here
x_train = np.random.rand(5, 588, 2160, 5)
y_train = np.random.randint(2, size=(num_samples, 1))

# define the model
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(588, 2160, 5)))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(5, activation='softmax'))

# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# fit the model
model.fit(x_train, y_train, epochs=10)

# evaluate the model
loss, accuracy = model.evaluate(x_train, y_train)


ValueError: Data cardinality is ambiguous:
  x sizes: 588
  y sizes: 5
Make sure all arrays contain the same number of samples.

In [21]:
# Define the input shape of your data
height, width, num_env_layers = 588, 2160, 5
input_shape = (height, width, num_env_layers + 1) # add 1 to account for the presence/absence layer

# Generate some dummy data for demonstration purposes
num_samples = 5
x_train = np.random.rand(num_samples, height, width, num_env_layers + 1)
y_train = np.random.randint(2, size=(num_samples, 1))

# Define your convolutional neural network model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile your model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train your model on your data
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

x_test = np.random.rand(num_samples, height, width, num_env_layers + 1)
y_test = np.random.randint(2, size=(num_samples, 1))

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

Epoch 1/10
1/1 [==============================] - 89s 89s/step - loss: 0.7695 - accuracy: 0.2500 - val_loss: 1883.5299 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 69s 69s/step - loss: 465.8575 - accuracy: 0.7500 - val_loss: 1544.6371 - val_accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 88s 88s/step - loss: 363.8896 - accuracy: 0.7500 - val_loss: 853.0097 - val_accuracy: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 125s 125s/step - loss: 180.1782 - accuracy: 0.7500 - val_loss: 293.4124 - val_accuracy: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 117s 117s/step - loss: 32.8325 - accuracy: 0.7500 - val_loss: 3.8201e-25 - val_accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 142s 142s/step - loss: 40.0741 - accuracy: 0.2500 - val_loss: 5.1740 - val_accuracy: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 147s 147s/step - loss: 8.6820e-06 - accuracy: 1.0000 - val_loss: 41.

In [42]:
y_prob = model.predict(x_test)
y_prob

1/1 [==============================] - 1s 981ms/step


array([[4.1648146e-08],
       [4.0686082e-08],
       [4.0467679e-08],
       [3.9664378e-08],
       [3.9999417e-08]], dtype=float32)

## Notes

### How to make a distribution model with CNNs
This paper [(Deneu et al. 2021)](https://doi.org/10.1371/journal.pcbi.1008856) first demonstrated using a CNN to predict the probability of a species presence in a grid cell. The input is a 2D grid of environmental variables and the output is several 2D grid of probabilities per species. 

## Issues

### CNN Input Dimensions/Shape
The suggested input shape for a CNN in Keras is (num_samples, height, width, layers). However, since we are trying to make a distribution model and not classification, there are no samples. There are only 2D grids of environmental variables and presence/absence layers. So, the input shape should be (height, width, layers). The code needs to be modified to work with distribution models.

